In [0]:
import gym
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import copy

In [5]:
env = gym.make("MountainCar-v0")

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
class MountainCar:
    def __init__(self, env):
        self.env = env

        self.gamma = 0.99
        self.lr = 0.001
        self.device = torch.device("cuda")

        self.episodes = 50000
        self.steps = 1000

        self.train_model, self.target_model, self.optimizer = self.create_model()
        self.memory_buffer = deque(maxlen=20000)
        self.batch = 32

        self.eps = 0.7
        self.eps_decay = 0.05

    def create_model(self):
        train_model = torch.nn.Sequential(
            torch.nn.Linear(2, 24),
            torch.nn.ReLU(),
            torch.nn.Linear(24, 48),
            torch.nn.ReLU(),
            torch.nn.Linear(48, 3),
        )
        target_model = copy.deepcopy(train_model)

        optimizer = torch.optim.Adam(train_model.parameters(), lr=self.lr)
    
        train_model.to(self.device)
        target_model.to(self.device)

        return train_model, target_model, optimizer


    def pick_best_action(self, state):
        return self.train_model(torch.tensor(state).to(self.device).float().unsqueeze(0))[0].max(0)[1].view(1, 1).item()

    def fit(self):
        if len(self.memory_buffer) < self.batch:
            return

        samples = random.sample(self.memory_buffer, self.batch)





    def run_episode(self, state):
        score = 0
        done = False

        while not done:
            # print(np.random.randint(3))
            action = np.random.randint(3) if np.random.random() < 0.9 else self.pick_best_action(state)
            state_, reward, done, _ = env.step(action)
            
            self.memory_buffer.append([state, action, reward, state_, done])

            self.fit()

            score += reward
            state = state_
            self.eps -= self.eps_decay


    def start_env(self):
        for ep in range(self.episodes):
            state = self.env.reset()
            self.run_episode(state)


In [65]:
car = MountainCar(env)
car.start_env()

KeyboardInterrupt: ignored